In [ ]:
# Copyright 2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deserialize (for Inference result)
This notebook explains the workflow for deserializing inference result. <br>
Instructions are described in [README.md](./README.md).

## Imports

In [ ]:
import base64
import errno
import json
import os
import subprocess
import tempfile
from pathlib import Path

import jsonschema

## Load Configurations
Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration
jsonschema.validate(json_load, json_schema)

## Check configuration parameters

In [ ]:
def check_file_exist(path: Path):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"File is Not Found. ({path})")


def get_relative_path(path: Path, base_dir: Path):
    resolve_path = path.resolve()
    resolve_base_dir = base_dir.resolve()
    relation_path = resolve_path.relative_to(resolve_base_dir)
    return relation_path


# check symlink
schema_file = Path(json_load["schema_file"].replace(os.path.sep, "/"))
validate_symlink(schema_file)
serialized_file = Path(json_load["serialized_file"].replace(os.path.sep, "/"))
validate_symlink(serialized_file)
output_dir = Path(json_load.get("output_dir", "./"))
validate_symlink(output_dir)

# File exist check
check_file_exist(schema_file)
check_file_exist(serialized_file)

# set relation path
base_dir = Path("./")
schema_file = get_relative_path(schema_file, base_dir)
serialized_file = get_relative_path(serialized_file, base_dir)
output_dir = get_relative_path(output_dir, base_dir)

# output_dir check
output_file = os.path.join(
    output_dir, os.path.splitext(os.path.basename(serialized_file))[0] + ".json"
)
if os.path.isfile(output_file):
    raise FileExistsError(f"File already exists. ({output_file})")

## Deserialize


In [ ]:
def decode(file_path: str):
    # Load configuration schema file
    with open(file_path, "r") as f:
        base64_file = json.load(f)
    # Get inference result
    if (
        "Inferences" in base64_file
        and len(base64_file["Inferences"]) > 0
        and "O" in base64_file["Inferences"][0]
    ):
        encoded_string = base64_file["Inferences"][0]["O"]
    else:
        raise ValueError('"Inferences" - "O" property doesn\'t exist in the json file')
    # Base64 decode
    buf_decode = base64.b64decode(encoded_string)

    return buf_decode


os.makedirs(output_dir, exist_ok=True)
with tempfile.TemporaryDirectory(dir=output_dir) as temp_binary_dir:
    if json_load["input_type"] == "json":
        buf = bytearray(decode(serialized_file))
        binary_file = os.path.join(
            temp_binary_dir,
            os.path.splitext(os.path.basename(serialized_file))[0] + ".bin",
        )
        with open(binary_file, "wb") as file:
            file.write(buf)
    else:
        binary_file = serialized_file

    proc = subprocess.run(
        ["./binary_to_json.sh", f"{schema_file}", f"{binary_file}", f"{output_dir}"],
        cwd=base_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )
    if proc.returncode != 0:
        raise ValueError(f"Deserialize Error: {proc.stderr}")
    else:
        print("file is deserialized.")
        print("Click the output file path below to view the deserialization results.")
        print(f"\toutput file path: ./{output_file}")